In [1]:
import numpy as np
import minimg as mg
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from sklearn.linear_model import LinearRegression
from utils import *
%matplotlib inline


In [ ]:
def get_rootpolyfeat(X):
    feat = np.zeros((X.shape[0], 6))
    feat[:,:3] = X
    feat[:, 3] = np.sqrt(X[:, 0] * X[:, 1])
    feat[:, 4] = np.sqrt(X[:, 1] * X[:, 2])
    feat[:, 5] = np.sqrt(X[:, 0] * X[:, 2])
    return feat

def fit_rootpoly(X, y):
    r_clsf = linear_model.LinearRegression()
    g_clsf = linear_model.LinearRegression()
    b_clsf = linear_model.LinearRegression()
    r_clsf.fit(get_rootpolyfeat(X), y[:, 0])
    g_clsf.fit(get_rootpolyfeat(X), y[:, 1])
    b_clsf.fit(get_rootpolyfeat(X), y[:, 2])
    A = np.zeros((3, 6))
    A[0] = r_clsf.coef_
    A[1] = g_clsf.coef_
    A[2] = b_clsf.coef_
    return A

def predict_rootpoly(A, img):
    img_t = img.copy()
    h = img_t.shape[0]
    l = img_t.shape[1]
    img_t = np.reshape((np.dot(A, (get_rootpolyfeat(np.reshape(img_t, (h * l, 3)))).T)).T, (h, l, 3))
    return img_t
    

In [ ]:
X = [[2, 2, 2], [1, 1, 5]]
y = [[2, 3, 1], [1, 1, 2]]
fit_rootpoly(np.array(X), np.array(y))

In [ ]:
calimg_T2_raw = np.float64(np.asarray(mg.load("./raw tiff/raw0081.tif")))
calimg_T2_gt = np.float64(np.asarray(mg.load("./gt tiff/gt0081.tif")))

maskT2 = np.loadtxt("./Canon EOS-1Ds Mark III/CHECKER/Canon1DsMkIII_0081_mask.txt", delimiter=',')
maskT2 *= 2 #размеры разные ¯\_(ツ)_/¯

T2 = fit_rootpoly(get_colors_from_patches(get_clrchkr(calimg_T2_raw, maskT2), maskT2),
          get_colors_from_patches(get_clrchkr(calimg_T2_gt, maskT2), maskT2))

print("T2:\n", T2)




calimg_T1_raw = np.float64(np.asarray(mg.load("./raw tiff/raw0118.tif")))
calimg_T1_gt = np.float64(np.asarray(mg.load("./gt tiff/gt0118.tif")))

maskT1 = np.loadtxt("./Canon EOS-1Ds Mark III/CHECKER/Canon1DsMkIII_0118_mask.txt", delimiter=',')
maskT1 *= 2

T1 = fit_rootpoly(get_colors_from_patches(get_clrchkr(calimg_T1_raw, maskT1), maskT1),
          get_colors_from_patches(get_clrchkr(calimg_T1_gt, maskT1), maskT1))

print("T1:\n", T1)

In [ ]:
RAW = ['./raw tiff/raw0015.tif', './raw tiff/raw0017.tif', './raw tiff/raw0024.tif', './raw tiff/raw0031.tif', './raw tiff/raw0057.tif', 
       './raw tiff/raw0065.tif', './raw tiff/raw0081.tif', './raw tiff/raw0118.tif', './raw tiff/raw0122.tif', './raw tiff/raw0154.tif']
GT = ['./gt tiff/gt0015.tif', './gt tiff/gt0017.tif', './gt tiff/gt0024.tif', './gt tiff/gt0031.tif', './gt tiff/gt0057.tif', 
       './gt tiff/gt0065.tif', './gt tiff/gt0081.tif', './gt tiff/gt0118.tif', './gt tiff/gt0122.tif', './gt tiff/gt0154.tif']
CCT = [5019.4815, 5656.3593, 5341.8299, 5784.7864, 5480.2725, 3424.9495, 5864.0833, 3024.8659, 5647.4098, 4510.6519]

In [ ]:
def pipeline(raw, cct, T1, T2, ccT1, ccT2):
    g = (1 / cct - 1 / ccT2) / (1 / ccT1 - 1 / ccT2)
    T = g * T1 + (1 - g) * T2
    #print(T)
    return predict_rootpoly(T, raw)    

In [ ]:
mean_before = []
mean_after = []

for i in tqdm_notebook(range(len(RAW))):
    img_raw = np.float64(np.asarray(mg.load(RAW[i])))
    img_gt = np.float64(np.asarray(mg.load(GT[i])))
    
    img_transformed = pipeline(img_raw, CCT[i], T1, T2, 3024.8659, 5864.0833)
    
    d_after = angdiff(img_transformed, img_gt)
    d_before = angdiff(img_raw, img_gt)
    
    print("======\nT=", CCT[i], " (calibrate temps: ", 3024.8659, ", ", 5864.0833, ")")
    print("MAE before transform: ", d_before.mean(), " max error: ", np.amax(d_before))
    print("MAE after transform: ", d_after.mean(), " max error: ", np.amax(d_after))
    
    mean_before.append(d_before.mean())
    mean_after.append(d_after.mean())
    
    
print(mean_after)
print(mean_before)

In [ ]:
RAW = ['./raw tiff/raw0015.tif', 
       './raw tiff/raw0017.tif', 
       './raw tiff/raw0024.tif', 
       './raw tiff/raw0031.tif', 
       './raw tiff/raw0057.tif', 
       './raw tiff/raw0065.tif', 
       #'./raw tiff/raw0081.tif', 
       './raw tiff/raw0118.tif', 
       './raw tiff/raw0122.tif', 
       './raw tiff/raw0154.tif']

GT = ['./gt tiff/gt0015.tif', 
      './gt tiff/gt0017.tif', 
      './gt tiff/gt0024.tif', 
      './gt tiff/gt0031.tif', 
      './gt tiff/gt0057.tif', 
      './gt tiff/gt0065.tif', 
      #'./gt tiff/gt0081.tif', 
      './gt tiff/gt0118.tif', 
      './gt tiff/gt0122.tif', 
      './gt tiff/gt0154.tif']

CCT = [5019.4815, 
       5656.3593, 
       5341.8299, 
       5784.7864,  #T2
       5480.2725, 
       3424.9495, 
       #5864.0833, 
       3024.8659, #T1
       5647.4098, 
       4510.6519]

In [ ]:
calimg_T2_raw = np.float64(np.asarray(mg.load("./raw tiff/raw0031.tif")))
calimg_T2_gt = np.float64(np.asarray(mg.load("./gt tiff/gt0031.tif")))

maskT2 = np.loadtxt("./Canon EOS-1Ds Mark III/CHECKER/Canon1DsMkIII_0031_mask.txt", delimiter=',')
maskT2 *= 2 #размеры разные ¯\_(ツ)_/¯

T2 = fit_rootpoly(get_colors_from_patches(get_clrchkr(calimg_T2_raw, maskT2), maskT2),
          get_colors_from_patches(get_clrchkr(calimg_T2_gt, maskT2), maskT2))

print("T2:\n", T2)



In [ ]:
mean_before = []
mean_after = []

for i in tqdm_notebook(range(len(RAW))):
    img_raw = np.float64(np.asarray(mg.load(RAW[i])))
    img_gt = np.float64(np.asarray(mg.load(GT[i])))
    
    img_transformed = pipeline(img_raw, CCT[i], T1, T2, 3024.8659, 5784.7864)
    
    d_after = angdiff(img_transformed, img_gt)
    d_before = angdiff(img_raw, img_gt)
    
    print("======\nT=", CCT[i], " (calibrate temps: ", 3024.8659, ", ", 5784.7864, ")")
    print("MAE before transform: ", d_before.mean(), " max error: ", np.amax(d_before))
    print("MAE after transform: ", d_after.mean(), " max error: ", np.amax(d_after))
    
    mean_before.append(d_before.mean())
    mean_after.append(d_after.mean())
    
    
print(mean_after)
print(mean_before)